In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else)","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc)",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist)",Name of the institute I graduated from last.,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you),Submitting this document means giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.,My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Video Introduction
0,Male,32,Mumbai,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,Certified financial analyst CFA,Masters,Health concious due to health conditions in fa...,I give consent,,,,,,
1,Male,36,New Delhi,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,Institute of chartered accountants of India,Masters,I like playing sports,I give consent,,,,,,
2,Male,32,Bangalore,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),"RRCE, Bangalore",Masters,No,I give consent,,,,,,
3,Female,33,Gurugram,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,PGI Rohtak,Phd and equivalent,I take time to open up and quite curious,I give consent,,,,,,
4,Female,32,Mumbai,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,XIC,PG Diploma,I believe honest communication and mutual resp...,I give consent,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,Male,32,Bangalore/ Chennai,Having people (family or friends) to share you...,"I had a typical Chennai upbringing, filled wit...",Was married earlier,No,Between 40-50 Lakhs,Marketing director,Great Lakes Insitute of Management,Masters,"I value open communication, honesty, and mutua...",I give consent,Hindu,Yes,It will not work for me,I am not sure,,
241,Male,37,Bangalore,Having a supportive family,It was good,Never married,No,Under 12 Lakhs,Journalist,Apex Professional University,Masters,Meet a person to know about them. Do not judge...,I give consent,Hindu,Yes,It will not work for me,No,,
242,Male,31,Delhi,am profoundly grateful for the unwavering sup...,"I spent my formative years in Kanpur, Uttar Pr...",Never married,No,Between 30-35 lakhs,As an educational professional based in the De...,IIT ISM Dhanbad,Masters,I am currently seeking a life partner within t...,I give consent,Hindu,I am not,It will not work for me,No,Friend recommended,
243,Female,32,Bangalore,The access and affordability to everything I n...,I grew up carefree and happy go lucky! However...,"Separated, filed for a divorce",No,Under 12 Lakhs,Counselling Psychologist,Christ University,Masters,"I love dancing, working out, travel, theater a...",I give consent,Spiritual,Yes,I am not sure,No,Friend recommended,


In [3]:
dataset.index = np.arange(2, len(dataset) + 2)
dataset

,My Gender,My age,My city of residence,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else)","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc)",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist)",Name of the institute I graduated from last.,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you),Submitting this document means giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.,My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Video Introduction
2,Male,32,Mumbai,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,Certified financial analyst CFA,Masters,Health concious due to health conditions in fa...,I give consent,,,,,,
3,Male,36,New Delhi,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,Institute of chartered accountants of India,Masters,I like playing sports,I give consent,,,,,,
4,Male,32,Bangalore,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),"RRCE, Bangalore",Masters,No,I give consent,,,,,,
5,Female,33,Gurugram,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,PGI Rohtak,Phd and equivalent,I take time to open up and quite curious,I give consent,,,,,,
6,Female,32,Mumbai,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,XIC,PG Diploma,I believe honest communication and mutual resp...,I give consent,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,Male,32,Bangalore/ Chennai,Having people (family or friends) to share you...,"I had a typical Chennai upbringing, filled wit...",Was married earlier,No,Between 40-50 Lakhs,Marketing director,Great Lakes Insitute of Management,Masters,"I value open communication, honesty, and mutua...",I give consent,Hindu,Yes,It will not work for me,I am not sure,,
243,Male,37,Bangalore,Having a supportive family,It was good,Never married,No,Under 12 Lakhs,Journalist,Apex Professional University,Masters,Meet a person to know about them. Do not judge...,I give consent,Hindu,Yes,It will not work for me,No,,
244,Male,31,Delhi,am profoundly grateful for the unwavering sup...,"I spent my formative years in Kanpur, Uttar Pr...",Never married,No,Between 30-35 lakhs,As an educational professional based in the De...,IIT ISM Dhanbad,Masters,I am currently seeking a life partner within t...,I give consent,Hindu,I am not,It will not work for me,No,Friend recommended,
245,Female,32,Bangalore,The access and affordability to everything I n...,I grew up carefree and happy go lucky! However...,"Separated, filed for a divorce",No,Under 12 Lakhs,Counselling Psychologist,Christ University,Masters,"I love dancing, working out, travel, theater a...",I give consent,Spiritual,Yes,I am not sure,No,Friend recommended,


In [4]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,5])

In [5]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
242,Male,32,Bangalore/ Chennai,Was married earlier
243,Male,37,Bangalore,Never married
244,Male,31,Delhi,Never married
245,Female,32,Bangalore,"Separated, filed for a divorce"


In [6]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [8]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [9]:
encoded_categorical_features

,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,Widow/ Widower
2,0,1,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,0,0,0,1,0
6,1,0,0,0,0
...,...,...,...,...,...
242,0,0,0,1,0
243,1,0,0,0,0
244,1,0,0,0,0
245,0,1,0,0,0


In [10]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [11]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
242,Male,32,Bangalore/ Chennai,Was married earlier
243,Male,37,Bangalore,Never married
244,Male,31,Delhi,Never married
245,Female,32,Bangalore,"Separated, filed for a divorce"


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [13]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
242,Male,32,Bangalore/ Chennai,Was married earlier
243,Male,37,Bangalore,Never married
244,Male,31,Delhi,Never married
245,Female,32,Bangalore,"Separated, filed for a divorce"


In [14]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [16]:
print(features.shape)

(245, 130)


In [17]:
k =100
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=100)

In [18]:
row =210
selectedRow = refDataset.iloc[row]

In [19]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[210 220 113 109 108  11  45 165  25  41  48 156  24  26 229 139  51 181
   54 110  81 123 136  84  30 122 231  19  86  55 128  42 183 118  97  32
  100 114 185  12 111  65 200  52  29 178  75  18 105  73  50  56 244 135
  237 196 169  36   1 160 209  66 121  96 166  77 221 228  27 174   6 184
  238 104  21 112 201  83  91 241 194  89 143 230 219  71 171 208 163  62
   76  37  59 106  35 173 142  53 214 119]]


In [20]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Male' 54 'Delhi ' 'Was married earlier']


['Male' 45 'Delhi ' 'Was married earlier']


['Male' 44 'Delhi' 'Was married earlier']


['Male' 49 'New Delhi ' 'Was married earlier']


['Male' 43 'Delhi' 'Was married earlier']


['Male' 37 'Delhi' 'Never married']


['Male' 42 'Bangalore ' 'Never married']


['Male' 42 'Mumbai' 'Never married']


['Male' 45 'New Delhi ' 'Was married earlier']


['Male' 38 'New Delhi' 'Never married']


['Male' 48 'Mumbai ' 'Separated, filed for a divorce']


['Male' 48 'Bangalore' 'Separated, not legally divorced']


['Male' 36 'Delhi' 'Never married']


['Male' 36 'Delhi ' 'Never married']


['Male' 40 'Hyderabad/pune' 'Never married']


['Male' 46 'Greater Noida' 'Separated, not legally divorced']


['Male' 42 'Delhi / NCR' 'Was married earlier']


['Male' 45 'Hyderabad Telangana India ' 'Separated, not legally divorced']


['Male' 35 'Delhi' 'Never married']


['Male' 39 'Bangalore' 'Never married']


['Male' 39 'Tampa USA ' 'Never married']


['Male'

In [21]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,5])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [22]:
print(preprocessData(refDataset))

[[ 1.         -0.89360542  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.14702971  1.         ...  0.          0.
   0.        ]
 [ 1.         -0.89360542  1.         ...  0.          0.
   0.        ]
 ...
 [ 1.         -1.08024935  1.         ...  0.          0.
   0.        ]
 [ 0.         -0.89360542  0.         ...  0.          0.
   0.        ]
 [ 0.          1.15947779  0.         ...  0.          0.
   0.        ]]


In [23]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']